In [ ]:
import config.setting
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, text

# Database connection setup
db_url = f"postgresql+psycopg2://{config.setting.db_user}:{config.setting.db_password}@{config.setting.db_host}/{config.setting.db_name}"
engine = create_engine(db_url)

# Fetch symbols from the Asset table in PostgreSQL
symbols = []
with engine.connect() as conn:
    result = conn.execute(text("SELECT AssetCode FROM Asset"))
    symbols = [row['AssetCode'] for row in result]

# Fetch and process data for each symbol
all_data = []

for symbol in symbols:
    try:
        df = yf.download(tickers=symbol, period="5d", interval="1d")
        if df.empty:
            print(f"No data for symbol: {symbol}")
            continue

        df['Symbol'] = symbol
        df.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
        df = df.round(2)
        df.reset_index(level=0, inplace=True)
        all_data.append(df)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

# Combine all data into a single DataFrame
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)

    # Insert data into pricestaging table
    table_name = 'pricestaging'
    combined_df.to_sql(name=table_name, con=engine, if_exists='append', index=False)

    # Execute stored procedure to insert new prices
    with engine.begin() as conn:
        conn.execute(text("CALL spInsertNewPrices();"))

print("Data fetch and insertion complete.")